In [1]:
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical


model_name = 'VGG16'  

if model_name == 'VGG16':
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
elif model_name == 'VGG19':
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
elif model_name == 'InceptionV3':
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [2]:

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(10, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=x)
for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [3]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()


x_train = x_train / 255.0
x_test = x_test / 255.0


y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


In [4]:

loss, accuracy_before = model.evaluate(x_test, y_test, verbose=2)
print(f'Accuracy before fine-tuning: {accuracy_before:.2f}')


I0000 00:00:1734550484.917170     164 service.cc:145] XLA service 0x7d4220004cb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734550484.917226     164 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1734550484.917231     164 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1734550487.624993     164 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 - 6s - 20ms/step - accuracy: 0.1092 - loss: 2.4674
Accuracy before fine-tuning: 0.11


In [5]:

for layer in base_model.layers[-10:]:  
    layer.trainable = True


model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 36s 37ms/step - accuracy: 0.6059 - loss: 1.1159 - val_accuracy: 0.7547 - val_loss: 0.7169
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.8065 - loss: 0.5616 - val_accuracy: 0.7992 - val_loss: 0.5897
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.8653 - loss: 0.3897 - val_accuracy: 0.8066 - val_loss: 0.5865
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9100 - loss: 0.2629 - val_accuracy: 0.7923 - val_loss: 0.6707
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9417 - loss: 0.1694 - val_accuracy: 0.8115 - val_loss: 0.6604
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9592 - loss: 0.1153 - val_accuracy: 0.8107 - val_loss: 0.7443
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9699 - loss: 0.0911 - val_accuracy: 0.8207 - val_loss: 0.7751
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 33ms/step - accuracy: 0.9757 - loss: 0.0735 - 

In [6]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f'Accuracy after fine-tuning: {accuracy :.2f}')


313/313 - 2s - 7ms/step - accuracy: 0.8202 - loss: 0.8227
Accuracy after fine-tuning: 0.82
